<a href="https://colab.research.google.com/github/AnnSenina/Python_for_CL/blob/main/notebooks/Python_12_pymorphy%2C_mystem_%D0%B4%D0%BE%D0%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лемматизация, морф.анализ

Для русского основых лемматизатора два: Pymorphy и Mystem.

##PyMorphy2

Не самый лучший в смысле точности, но очень популярный и простой инструмент для лемматизации

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.1 MB/s 
     |████████████████████████████████| 8.2 MB 10.9 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=4f672b17528447cd8b01a0751dd163c72d3a704e51dc026d1f0a449f0f1f5374
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

### основная функция - pymorphy.parse
     
Она похожа на analyze в mystem, возвращает список объектов Parse

Первый в списке - самый вероятный разбор (у каждого есть score)

Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать

In [ ]:
morph.parse('человек')

[Parse(word='человек', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='человек', score=0.569458, methods_stack=((DictionaryAnalyzer(), 'человек', 3166, 0),)),
 Parse(word='человек', tag=OpencorporaTag('NOUN,anim,masc plur,gent'), normal_form='человек', score=0.430541, methods_stack=((DictionaryAnalyzer(), 'человек', 3166, 8),))]

Документация [здесь](https://pymorphy2.readthedocs.io/en/stable/user/guide.html#id3), обозначения для граммем [здесь](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs)

In [ ]:
# сделаем красиво
print('Cлово - ', morph.parse('человеком')[0].word)
print('Лемма слова - ', morph.parse('человеком')[0].normal_form)
print('Грамматическая информация слова - ', morph.parse('человеком')[0].tag)
print('Часть речи слова - ', morph.parse('человеком')[0].tag.POS)
print('Род слова - ', morph.parse('человеком')[0].tag.gender)
print('Число слова - ', morph.parse('человеком')[0].tag.number)
print('Падеж слова - ', morph.parse('человеком')[0].tag.case)

Cлово -  человеком
Лемма слова -  человек
Грамматическая информация слова -  NOUN,anim,masc sing,ablt
Часть речи слова -  NOUN
Род слова -  masc
Число слова -  sing
Падеж слова -  ablt


In [ ]:
# для лемматизации нужна именно эта команда
morph.parse('человеком')[0].normal_form

'человек'

In [ ]:
morph.parse('сибил')

[Parse(word='сибил', tag=OpencorporaTag('VERB,impf,tran masc,sing,past,indc'), normal_form='сибить', score=0.5060240963855422, methods_stack=((DictionaryAnalyzer(), 'бил', 465, 7), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'си'))),
 Parse(word='сибил', tag=OpencorporaTag('VERB,perf,tran masc,sing,past,indc'), normal_form='сибить', score=0.4578313253012049, methods_stack=((FakeDictionary(), 'сибил', 477, 1), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'бил'))),
 Parse(word='сибил', tag=OpencorporaTag('NOUN,inan,neut plur,gent'), normal_form='сибило', score=0.03614457831325302, methods_stack=((FakeDictionary(), 'сибил', 54, 7), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'бил')))]

Давайте снова возьмем Сэлинджера...

Увы! pymorphy не умеет токенизировать, поэтому...

In [ ]:
# все то, что надо импортировать из NLTK
#!pip install nltk

import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import download
download('punkt')
download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# собрала препроцессинг в 1 функцию
def clean_text(text_str):
  text_str = text_str.lower() # нижний регистр
  text_list_nltk = word_tokenize(text_str) # токенизация
  stop_words = stopwords.words('russian') 
  text_without_punkt = [word for word in text_list_nltk if word[0].isalpha()] # удалить пунктуацию из списка токенов
  text_clean = [word for word in text_without_punkt if word not in stop_words] # чистим от стоп-слов
  return text_clean

with open('bananafish.txt', 'r', encoding='utf-8') as file:
  bananafish = file.read()
bananafish = bananafish.replace('\ufeff', '')
bananafish = bananafish.replace('\xa0', ' ')
bananafish_tokens = clean_text(bananafish)
bananafish_tokens

['джером',
 'дэвид',
 'сэлинджер',
 'девять',
 'рассказов',
 'самый',
 'день',
 'банабульки',
 'отеле',
 'поселились',
 'девяносто',
 'семь',
 'рекламщиков',
 'нью‑йорка',
 'забили',
 'междугородные',
 'линии',
 'девушке',
 'пришлось',
 'ждать',
 'соединения',
 'полудня',
 'половины',
 'третьего',
 'однако',
 'времени',
 'теряла',
 'прочла',
 'женском',
 'карманном',
 'журнале',
 'статью',
 'секс',
 'развлечение',
 'ад',
 'вымыла',
 'щетку',
 'расческу',
 'свела',
 'небольшое',
 'пятно',
 'юбки',
 'бежевого',
 'костюма',
 'перешила',
 'пуговицу',
 'блузке',
 'сакса',
 'выдернула',
 'свежих',
 'волоска',
 'пробившихся',
 'родинки',
 'телефонистка',
 'вызвала',
 'девушкин',
 'номер',
 'та',
 'сидела',
 'нише',
 'окна',
 'докрашивала',
 'ногти',
 'левой',
 'руке',
 'такие',
 'девушки',
 'совершенно',
 'станут',
 'бросать',
 'звонит',
 'телефон',
 'таких',
 'судя',
 'всему',
 'телефон',
 'умолкает',
 'наступления',
 'половозрелости',
 'телефон',
 'разрывался',
 'подвела',
 'кисточку',
 'но

In [ ]:
words_lemmatized = []
for word in bananafish_tokens:
  if word == 'сибил':
    words_lemmatized.append(word)
  else:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    words_lemmatized.append(normal_form)

words_lemmatized
# print(words_lemmatized)

In [ ]:
words_lemmatized = []
for word in bananafish_tokens:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    words_lemmatized.append(normal_form)

words_lemmatized
# print(words_lemmatized)

['джером',
 'дэвид',
 'сэлинджер',
 'девять',
 'рассказ',
 'самый',
 'день',
 'банабулька',
 'отель',
 'поселиться',
 'девяносто',
 'семь',
 'рекламщик',
 'нью‑йорк',
 'забить',
 'междугородный',
 'линия',
 'девушка',
 'прийтись',
 'ждать',
 'соединение',
 'полдень',
 'половина',
 'третий',
 'однако',
 'время',
 'терять',
 'прочесть',
 'женский',
 'карманный',
 'журнал',
 'статья',
 'секс',
 'развлечение',
 'ад',
 'вымыть',
 'щётка',
 'расчёска',
 'свести',
 'небольшой',
 'пятно',
 'юбка',
 'бежевый',
 'костюм',
 'перешить',
 'пуговица',
 'блузка',
 'сакс',
 'выдернуть',
 'свежий',
 'волосок',
 'пробиться',
 'родинка',
 'телефонистка',
 'вызвать',
 'девушкина',
 'номер',
 'тот',
 'сидеть',
 'ниша',
 'окно',
 'докрашивать',
 'ноготь',
 'левый',
 'рука',
 'такой',
 'девушка',
 'совершенно',
 'стать',
 'бросать',
 'звонить',
 'телефон',
 'такой',
 'судить',
 'весь',
 'телефон',
 'умолкать',
 'наступление',
 'половозрелость',
 'телефон',
 'разрываться',
 'подвести',
 'кисточка',
 'ноготь',

In [ ]:
for i in range(len(words_lemmatized)): # по индексам
  if words_lemmatized[i] == 'сибить':
    words_lemmatized[i] = 'сибил'

In [ ]:
from collections import Counter

Counter(words_lemmatized).most_common(10)
#print(Counter(words_lemmatized).most_common(10))

[('сказать', 72),
 ('сибил', 52),
 ('девушка', 34),
 ('человек', 30),
 ('молодой', 29),
 ('мама', 24),
 ('спросить', 19),
 ('это', 18),
 ('ответить', 17),
 ('знать', 16)]

In [ ]:
import nltk

freq_bigramms = Counter(nltk.bigrams(words_lemmatized))
freq_bigramms.most_common(10)
#print(Counter(words_lemmatized).most_common(10))

# что-то пошло не так?

[(('молодой', 'человек'), 29),
 (('сказать', 'девушка'), 14),
 (('сказать', 'молодой'), 10),
 (('сказать', 'сибил'), 10),
 (('ответить', 'девушка'), 9),
 (('вчера', 'вечером'), 8),
 (('спросить', 'сибил'), 7),
 (('шэрон', 'липшуц'), 7),
 (('мама', 'сказать'), 5),
 (('откуда', 'знать'), 4)]

#Более умный лемматизатор MyStem

Сравнительная точность pymorphy и mystem для русского языка [здесь](http://web-corpora.net/wsgi/mystemplus.wsgi/mystemplus/compare_table/)


In [ ]:
!pip install pymystem3 # Jypyter, PyCharm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Только для Colab!

In [ ]:
!pip install pymystem3==0.1.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pymystem3
    Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2022-12-10 09:42:11--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.244, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cachev2-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=235 [following]
--2022-12-10 09:42:11--  http://cachev2-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=235
Resolving cachev2-mskm906.cdn.yandex.net (cachev2-mskm906.cdn.yandex.net)... 5.45.220.117, 2a02:6b8:0:2002::917
Connecting to cachev2-mskm906.cdn.yandex.net (cachev2-mskm906.cdn.yandex.net)|5.45.220.117|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.

Дальше уже для всех!

In [ ]:
from pymystem3 import Mystem
mystem = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [ ]:
mystem.lemmatize('mystem даже сам! умеет токенизировать текст')

['mystem',
 ' ',
 'даже',
 ' ',
 'сам',
 '!',
 ' ',
 'уметь',
 ' ',
 'токенизировать',
 ' ',
 'текст',
 '\n']

In [ ]:
bananafish_tokens

In [ ]:
words_lemmatized_mystem = mystem.lemmatize(' '.join(bananafish_tokens))
words_lemmatized_mystem
#print(words_lemmatized_mystem)

['джером',
 ' ',
 'дэвид',
 ' ',
 'сэлинджер',
 ' ',
 'девять',
 ' ',
 'рассказ',
 ' ',
 'самый',
 ' ',
 'день',
 ' ',
 'банабулька',
 ' ',
 'отель',
 ' ',
 'поселяться',
 ' ',
 'девяносто',
 ' ',
 'семь',
 ' ',
 'рекламщик',
 ' ',
 'нью',
 '‑',
 'йорк',
 ' ',
 'забивать',
 ' ',
 'междугородный',
 ' ',
 'линия',
 ' ',
 'девушка',
 ' ',
 'приходиться',
 ' ',
 'ждать',
 ' ',
 'соединение',
 ' ',
 'полдень',
 ' ',
 'половина',
 ' ',
 'третий',
 ' ',
 'однако',
 ' ',
 'время',
 ' ',
 'терять',
 ' ',
 'прочитывать',
 ' ',
 'женский',
 ' ',
 'карманный',
 ' ',
 'журнал',
 ' ',
 'статья',
 ' ',
 'секс',
 ' ',
 'развлечение',
 ' ',
 'ад',
 ' ',
 'вымывать',
 ' ',
 'щетка',
 ' ',
 'расческа',
 ' ',
 'сводить',
 ' ',
 'небольшой',
 ' ',
 'пятно',
 ' ',
 'юбка',
 ' ',
 'бежевый',
 ' ',
 'костюм',
 ' ',
 'перешивать',
 ' ',
 'пуговица',
 ' ',
 'блузка',
 ' ',
 'сакс',
 ' ',
 'выдергивать',
 ' ',
 'свежий',
 ' ',
 'волосок',
 ' ',
 'пробиваться',
 ' ',
 'родинка',
 ' ',
 'телефонистка',
 ' ',
 'выз

In [ ]:
lem_text = ''.join(words_lemmatized_mystem)
lem_text

In [ ]:
from collections import Counter

Counter(words_lemmatized_mystem).most_common(10)
#print(Counter(words_lemmatized_mystem).most_common(10))

[(' ', 1910),
 ('сказать', 73),
 ('сибивать', 52),
 ('девушка', 34),
 ('‑', 30),
 ('человек', 30),
 ('молодой', 29),
 ('…', 26),
 ('мама', 24),
 ('спрашивать', 23)]

### морф. анализ в MyStem

In [ ]:
sometext = "Задача NLI важна для компьютерных лингвистов"
analyzed = mystem.analyze(sometext) 
print(analyzed) 

[{'analysis': [{'lex': 'задача', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}], 'text': 'Задача'}, {'text': ' '}, {'analysis': [], 'text': 'NLI'}, {'text': ' '}, {'analysis': [{'lex': 'важный', 'wt': 1, 'gr': 'A=ед,кр,жен'}], 'text': 'важна'}, {'text': ' '}, {'analysis': [{'lex': 'для', 'wt': 1, 'gr': 'PR='}], 'text': 'для'}, {'text': ' '}, {'analysis': [{'lex': 'компьютерный', 'wt': 1, 'gr': 'A=(пр,мн,полн|вин,мн,полн,од|род,мн,полн)'}], 'text': 'компьютерных'}, {'text': ' '}, {'analysis': [{'lex': 'лингвист', 'wt': 1, 'gr': 'S,муж,од=(вин,мн|род,мн)'}], 'text': 'лингвистов'}, {'text': '\n'}]


In [ ]:
# возвращает список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# wt - вес, увереноость в правильности
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
print('Слово - ', analyzed[0]['text'])
print('Разбор слова - ', analyzed[0]['analysis'][0])
print('Лемма слова - ', analyzed[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', analyzed[0]['analysis'][0]['gr'])

Слово -  Задача
Разбор слова -  {'lex': 'задача', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}
Лемма слова -  задача
Грамматическая информация слова -  S,жен,неод=им,ед


## Задание

Попробуйте сами придумать предложение с омонимами и сравнить морф.анализ через pymorphy и mystem

In [ ]:
text = 'Гуляя в мужской парке по парку им.Карима Тинчурина в Казане, я увидел много белок. Я кармил их куриными яичками, но они не ели белок.'

In [ ]:
text = clean_text(text)

### Варианты решений в группе

Сравните лемматизацию - лемматизируйте предложение с морф. неоднозначностью

In [ ]:
text = "Гуляя в мужской парке по парку им.Карима Тинчурина в Казане, я увидел много белок. Я кармил их куриными яичками, но они не ели белок."


def clean_text(text_str):
  text_str = text_str.lower() # нижний регистр
  text_list_nltk = word_tokenize(text_str) # токенизация
  stop_words = stopwords.words('russian') 
  text_without_punkt = [word for word in text_list_nltk if word[0].isalpha()] # удалить пунктуацию из списка токенов
  text_clean = [word for word in text_without_punkt if word not in stop_words] # чистим от стоп-слов
  return text_clean


text_tokens = clean_text(text)
print(text_tokens)

f_words_lemmatized = []
for word in text_tokens:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    f_words_lemmatized.append(normal_form)

f_words_lemmatized



['гуляя', 'мужской', 'парке', 'парку', 'им.карима', 'тинчурина', 'казане', 'увидел', 'белок', 'кармил', 'куриными', 'яичками', 'ели', 'белок']


['гулять',
 'мужской',
 'парка',
 'парка',
 'им.карим',
 'тинчурина',
 'казан',
 'увидеть',
 'белок',
 'кармила',
 'куриный',
 'яичко',
 'есть',
 'белок']

In [ ]:
# Mystem

s_words_lemmatized_mystem = mystem.lemmatize(' '.join(text_tokens))
s_words_lemmatized_mystem
#print(words_lemmatized_myste) 

['гулять',
 ' ',
 'мужской',
 ' ',
 'парк',
 ' ',
 'парк',
 ' ',
 'он',
 '.',
 'карима',
 ' ',
 'тинчурин',
 ' ',
 'казан',
 ' ',
 'увидеть',
 ' ',
 'белок',
 ' ',
 'кармила',
 ' ',
 'курин',
 ' ',
 'яичко',
 ' ',
 'ель',
 ' ',
 'белок',
 '\n']

In [ ]:
text = "Гуляя в мужской парке по парку им.Карима Тинчурина в Казане, я увидел много белок. Я кармил их куриными яичками, но они не ели белок."

In [ ]:
print(' '.join(f_words_lemmatized))

гулять мужской парка парка им.карим тинчурина казан увидеть белок кармила куриный яичко есть белок


In [ ]:
print(''.join(s_words_lemmatized_mystem))

гулять мужской парк парк он.карима тинчурин казан увидеть белок кармила курин яичко ель белок



In [ ]:
t_words_lemmatized_mystem = mystem.lemmatize(text) 
t_words_lemmatized_mystem

['гулять',
 ' ',
 'в',
 ' ',
 'мужской',
 ' ',
 'парк',
 ' ',
 'по',
 ' ',
 'парк',
 ' ',
 'они',
 '.',
 'карима',
 ' ',
 'тинчурин',
 ' ',
 'в',
 ' ',
 'казан',
 ', ',
 'я',
 ' ',
 'увидеть',
 ' ',
 'много',
 ' ',
 'белок',
 '. ',
 'я',
 ' ',
 'кармила',
 ' ',
 'они',
 ' ',
 'куриный',
 ' ',
 'яичко',
 ', ',
 'но',
 ' ',
 'они',
 ' ',
 'не',
 ' ',
 'есть',
 ' ',
 'белок',
 '.',
 '\n']

In [ ]:
text ='Гуляя в мужской парке по парку им.Карима Тинчурина в Казане, я увидел много белок. Я кармил их куриными яичками, но они не ели белок.'
text = text.lower() # нижний регистр
text_list_nltk = word_tokenize(text) # токенизация
stop_words = stopwords.words('russian') 
text_without_punkt = [word for word in text_list_nltk if word[0].isalpha()] # удалить пунктуацию из списка токенов
text_clean = [word for word in text_without_punkt if word not in stop_words] # чистим от стоп-слов
words_lemmatized = []
for word in text_clean:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    words_lemmatized.append(normal_form)

words_lemmatized

['гулять',
 'мужской',
 'парка',
 'парка',
 'им.карим',
 'тинчурина',
 'казан',
 'увидеть',
 'белок',
 'кармила',
 'куриный',
 'яичко',
 'есть',
 'белок']

In [ ]:
text ='Гуляя в мужской парке по парку им.Карима Тинчурина в Казане, я увидел много белок. Я кармил их куриными яичками, но они не ели белок.'
text = text.lower() # нижний регистр
text_list_nltk = word_tokenize(text) # токенизация
stop_words = stopwords.words('russian') 
text_without_punkt = [word for word in text_list_nltk if word[0].isalpha()] # удалить пунктуацию из списка токенов
text_clean = [word for word in text_without_punkt if word not in stop_words] # чистим от стоп-слов
words_lemmatized = []
for word in text_clean:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    words_lemmatized.append(normal_form)


words_lemmatized_mystem = mystem.lemmatize(' '.join(text_clean))
print(words_lemmatized, words_lemmatized_mystem, end='/n')

['гулять', 'мужской', 'парка', 'парка', 'им.карим', 'тинчурина', 'казан', 'увидеть', 'белок', 'кармила', 'куриный', 'яичко', 'есть', 'белок'] ['гулять', ' ', 'мужской', ' ', 'парк', ' ', 'парк', ' ', 'он', '.', 'карима', ' ', 'тинчурин', ' ', 'казан', ' ', 'увидеть', ' ', 'белок', ' ', 'кармила', ' ', 'курин', ' ', 'яичко', ' ', 'ель', ' ', 'белок', '\n']/n

In [ ]:
text ='Гуляя в мужской парке по парку им.Карима Тинчурина в Казане, я увидел много белок. Я кармил их куриными яичками, но они не ели белок.'
text = text.lower() # нижний регистр
text_list_nltk = word_tokenize(text) # токенизация
stop_words = stopwords.words('russian') 
text_without_punkt = [word for word in text_list_nltk if word[0].isalpha()] # удалить пунктуацию из списка токенов
text_clean = [word for word in text_without_punkt if word not in stop_words] # чистим от стоп-слов
words_lemmatized = []
for word in text_clean:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    words_lemmatized.append(normal_form)

words_lemmatized_mystem = mystem.lemmatize(' '.join(text_clean))
words_lemmatized_mystem = sorted(words_lemmatized_mystem)[15:]

print(sorted(words_lemmatized))
print(words_lemmatized_mystem)

['белок', 'белок', 'гулять', 'есть', 'им.карим', 'казан', 'кармила', 'куриный', 'мужской', 'парка', 'парка', 'тинчурина', 'увидеть', 'яичко']
['белок', 'белок', 'гулять', 'ель', 'казан', 'карима', 'кармила', 'курин', 'мужской', 'он', 'парк', 'парк', 'тинчурин', 'увидеть', 'яичко']


### Доп. задание

Попробуйте написать новую функцию, которая принимает на вход текст как строку и возвращает список лемматизированных слов. После этого проверьте на тексте

In [ ]:
# ваша функция


Существуют и другие библиотеки для лемматизации

русский язык:

- [RNNmorph](https://github.com/IlyaGusev/rnnmorph)
- [deeppavlov](http://docs.deeppavlov.ai/en/master/)

другие языки:
- [UralicNLP](https://github.com/mikahama/uralicNLP)
- [hfst от Apertium](https://wiki.apertium.org/wiki/Hfst)
- [Stanza](https://stanfordnlp.github.io/stanza/)
- [SpaCy](https://spacy.io/usage/linguistic-features#morphology)
- [Trankit](https://trankit.readthedocs.io/en/latest/posdep.html)